In [2]:
!python3 --version

Python 3.7.13


In [1]:
# basic libraries
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
import matplotlib.pyplot as plt

# n-gram
import math
from collections import Counter, defaultdict
from typing import List # somehow python 3.9 requires

import nltk
import numpy as np
import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer
# from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm


# keras lstm model
from keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

"""
Config
跑之前要改config = writing_patterns/Quantity/Both

"""

writing_patterns = ['title_special_chars', 'text_special_chars',
       'title_dets', 'text_dets', 'title_capitals', 'text_capitals',
       'title_short_sents', 'text_short_sents', 'title_long_sents',
       'text_long_sents']

Quantity = ['title_words', 'text_words',
       'title_sents', 'text_sents', 'title_words_per_sent',
       'text_words_per_sent', 'title_verbs', 'text_verbs', 'title_adjs',
       'text_adjs', 'title_advs', 'text_advs', 'title_rate_adjs_and_advs',
       'text_rate_adjs_and_advs','title_fog', 'text_fog', 'title_smog', 'text_smog',
       'title_ari', 'text_ari']

Both = ['title_words', 'text_words',
       'title_sents', 'text_sents', 'title_words_per_sent',
       'text_words_per_sent', 'title_verbs', 'text_verbs', 'title_adjs',
       'text_adjs', 'title_advs', 'text_advs', 'title_rate_adjs_and_advs',
       'text_rate_adjs_and_advs', 'title_special_chars', 'text_special_chars',
       'title_dets', 'text_dets', 'title_capitals', 'text_capitals',
       'title_short_sents', 'text_short_sents', 'title_long_sents',
       'text_long_sents', 'title_fog', 'text_fog', 'title_smog', 'text_smog',
       'title_ari', 'text_ari']


config = writing_patterns


"""
Evaluation functions


"""

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def mcc_metric(y_true, y_pred):
  y_pred_pos = K.round(K.clip(y_pred, 0, 1))
  y_pred_neg = 1 - y_pred_pos

  y_pos = K.round(K.clip(y_true, 0, 1))
  y_neg = 1 - y_pos

  tp = K.sum(y_pos * y_pred_pos)
  tn = K.sum(y_neg * y_pred_neg)

  fp = K.sum(y_neg * y_pred_pos)
  fn = K.sum(y_pos * y_pred_neg)

  numerator = (tp * tn - fp * fn)
  denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

  return numerator / (denominator + K.epsilon())




In [2]:

"""
get Dataset from Drive

"""

from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive/') # 此處需要登入google帳號
!unzip -u "/content/gdrive/MyDrive/人工智慧概論23組/dataset (with valid).zip" -d "/content/"

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('valid.csv')


Mounted at /content/gdrive/
Archive:  /content/gdrive/MyDrive/人工智慧概論23組/dataset (with valid).zip
  inflating: /content/test.csv       
  inflating: /content/train.csv      
  inflating: /content/valid.csv      


In [24]:
class Ngram:
    def __init__(self, config, n=2):
        self.tokenizer = ToktokTokenizer()
        self.n = n
        self.model = None
        self.config = config
        self.counts = None # the counts for corpus


    def tokenize(self, sentence):
        return self.tokenizer.tokenize(sentence)

    #turning words that appeared less than 10/1 times as [UNK], num=10 or 2
    def turnToUnk(self, corpus, num):
        corpus_list = [j for i in corpus for j in i]
        counts = Counter(corpus_list)
        pdCounts = pd.DataFrame(counts.values(),index = list(counts))
        lessWords = list(pdCounts[pdCounts[0]<num].index)
        pd_corpus = pd.DataFrame(corpus_list)
        pd_corpus[pd_corpus[0].isin(lessWords)] = '[UNK]'
        pd_index = (pd_corpus[pd_corpus[0].str.contains('[CLS]',na=False)])
        length = len(pd_index.index)
        corpus = []
        for i,f in enumerate(pd_index.index):
            if(i == length-1):
                break
            ff = pd_index.index[i+1]
            corpus.append(list(pd_corpus[f:ff][0]))
        return corpus

    def get_ngram(self, corpus_tokenize: List[List[str]]):
        # begin your code (Part 1)
        
        bigram_tokens = [(x, i[j + 1]) for i in corpus_tokenize 
            for j, x in enumerate(i) if j < len(i) - 1]
        bigrams_freq = Counter(bigram_tokens)

        prob = defaultdict(lambda: defaultdict(lambda: 0))
        counts = defaultdict(lambda: defaultdict(lambda :0))
        for i in bigrams_freq:
            counts[i[0]][i[1]] = bigrams_freq[i]
        for i1 in counts:
            total_count = float(sum(counts[i1].values()))
            for i2 in counts[i1]:
                prob[i1][i2] = counts[i1][i2]/ total_count

        return prob, counts
    
    def train(self, df):
        '''
        Train n-gram model.
        '''
        corpus = [['[CLS]'] + self.tokenize(document) for document in df['text']] + [['[CLS]'] + self.tokenize(document) for document in df['title']] 
        # [CLS] represents start of sequence
        
        self.model, self.counts = self.get_ngram(corpus)
                


In [25]:
# 主程式
biGram = Ngram(2)
biGram.train(train)

In [26]:
"""
pick the first n features
pass to a DataFrame and sort by their frequency

"""

feature_num = 800
feature_counter = []
for i in biGram.counts:
    for r in biGram.counts[i]:
        feature_counter.append((i+" "+r,biGram.counts[i][r]))

df = pd.DataFrame(feature_counter)
df = df.sort_values(by=[1],ascending=False)
feature = list(df.iloc[0:feature_num,0])

In [27]:

x_train = []
x_test = []
x_valid = []


for i,f in enumerate(train['text']):
    corpus = [['[CLS]'] + biGram.tokenize(f)] + [['[CLS]'] + biGram.tokenize(train['title'][i])]

    bigram_tokens = [(x+" "+i[j + 1]) for i in corpus 
                for j, x in enumerate(i) if j < len(i) - 1]
    count = [0]* feature_num
    for f in bigram_tokens:
        if(f in feature):
            count[feature.index(f)]+=1
    x_train.append(count)



for i,f in enumerate(test['text']):
    corpus = [['[CLS]'] + biGram.tokenize(f)] + [['[CLS]'] + biGram.tokenize(test['title'][i])]
    
    bigram_tokens = [(x+" "+i[j + 1]) for i in corpus 
                for j, x in enumerate(i) if j < len(i) - 1]
    count = [0]*feature_num
    for f in bigram_tokens:
        if(f in feature):
            count[feature.index(f)]+=1
    x_test.append(count)

    
for i,f in enumerate(valid['text']):
    corpus = [['[CLS]'] + biGram.tokenize(f)] + [['[CLS]'] + biGram.tokenize(valid['title'][i])]
    
    bigram_tokens = [(x+" "+i[j + 1]) for i in corpus 
                for j, x in enumerate(i) if j < len(i) - 1]
    count = [0]*feature_num
    for f in bigram_tokens:
        if(f in feature):
            count[feature.index(f)]+=1
    x_valid.append(count)


In [28]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_valid = np.array(x_valid)


y_train = train['label']
y_val = valid['label']
y_test = test['label']

從這邊開始是LSTM

In [29]:
lstm_out = 64

model = Sequential()
model.add(Embedding(800, output_dim=256))
model.add(LSTM(lstm_out))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, name='out_layer'))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='adam',\
                metrics = ['accuracy', recall_m, precision_m, mcc_metric])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         204800    
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [30]:
history = model.fit(x_train, y_train, validation_data=(x_valid, y_val), epochs=20, batch_size=128)


Epoch 1/20
310/310 [==============================] - 23s 50ms/step - loss: 0.6936 - accuracy: 0.4999 - recall_m: 0.4514 - precision_m: 0.5006 - mcc_metric: 0.0015 - val_loss: 0.6931 - val_accuracy: 0.4999 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_mcc_metric: 0.0000e+00
Epoch 2/20
310/310 [==============================] - 15s 49ms/step - loss: 0.6938 - accuracy: 0.5030 - recall_m: 0.5531 - precision_m: 0.4680 - mcc_metric: 0.0070 - val_loss: 0.6936 - val_accuracy: 0.4999 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_mcc_metric: 0.0000e+00
Epoch 3/20
310/310 [==============================] - 15s 48ms/step - loss: 0.6926 - accuracy: 0.5113 - recall_m: 0.5785 - precision_m: 0.5070 - mcc_metric: 0.0295 - val_loss: 0.6943 - val_accuracy: 0.4999 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_mcc_metric: 0.0000e+00
Epoch 4/20
310/310 [==============================] - 15s 50ms/step - loss: 0.6884 - accuracy: 0.5320 - recall_m: 0.5714 - p

In [31]:
model.evaluate(x_test, y_test)

355/355 [==============================] - 6s 16ms/step - loss: 0.6146 - accuracy: 0.6437 - recall_m: 0.6932 - precision_m: 0.6293 - mcc_metric: 0.2893


[0.6145535707473755,
 0.6437461376190186,
 0.693248450756073,
 0.6292709708213806,
 0.28930288553237915]